In [48]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from PIL import Image, ImageOps, ImageEnhance
import glob
import shutil
import sys

In [49]:
# 変更する部分

In [64]:
# 入力とする画像のフォルダを指定。種類までのパスをコピペしてください。
# すべての種類を一気にやりたい場合は*\imageまでのパスをコピペしてください
#inputImageFolderPath = '..\学習データ\混合データ\品種\image'
inputImageFolderPath = r'\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ'

# すべての種類を一気にやる場合はTrueに、1つの種類のみやる場合はFalseにしてください。
#allFruits = True
allFruits = False

# 何の画像処理をさせるかを決めてください。前から順に処理されます。
# 0:彩度変更, 1:コントラスト変更, 2:明度変更, 3:シャープネス変更, 4:拡大・縮小, 5:上下反転, 6:左右反転, 7:90度回転, 8:270度回転
processList = [5]


In [65]:
# 変更しない部分

In [66]:
# 出力のフォルダを指定。いじらなくていいはずです。
outputRoot = '..\学習データ\水増しデータ\混合データ'
# 入力テキストフォルダ。いじらない。
inputTextFolderPath = inputImageFolderPath.replace('image', 'annotation')

# 彩度調整パラメータ。1.0以上で明るく、1.0未満で暗く
saturationRatio = 1.5

# コントラスト調整パラメータ。1.0以上で明るく、1.0未満で暗く
contrastRatio = 1.5

# 明度調整パラメータ。1.0以上で明るく、1.0未満で暗く
brightnessRatio = 1.5

# シャープネス調整パラメータ。1.0以上でくっきり、1.0未満でぼやける
sharpnessRatio = 1.5

# 大きさ調整パラメータ。縦横がこの倍率で大きくなります。
expansionRatio = 1.2

In [67]:
# 各種関数

In [68]:
def makeFolder(root, folderName, imageFile):
    outputPath = os.path.join(root, folderName)
    if not os.path.exists(outputPath):
        os.mkdir(outputPath)
    
    
    outputPath = os.path.join(outputPath, imageFile.split('\\')[-5])
    if not os.path.exists(outputPath):
        os.mkdir(outputPath)
    
    outputAnnotationPath = os.path.join(outputPath, "annotation")
    if not os.path.exists(outputAnnotationPath):
        os.mkdir(outputAnnotationPath)
    outputImagePath = os.path.join(outputPath, "image")
    if not os.path.exists(outputImagePath):
        os.mkdir(outputImagePath)
    
    imagePathList = imageFile.split('\\')
    
    outputAnnotationPath = os.path.join(outputAnnotationPath, imagePathList[-3])
    if not os.path.exists(outputAnnotationPath):
        os.mkdir(outputAnnotationPath)
    outputImagePath = os.path.join(outputImagePath, imagePathList[-3])
    if not os.path.exists(outputImagePath):
        os.mkdir(outputImagePath)
    
    outputAnnotationPath = os.path.join(outputAnnotationPath, imagePathList[-2])
    if not os.path.exists(outputAnnotationPath):
        os.mkdir(outputAnnotationPath)
        outputAnnotationPath = os.path.join(outputAnnotationPath, 'classes.txt')
        with open(outputAnnotationPath, mode = 'w') as f:
            f.write("")
    outputImagePath = os.path.join(outputImagePath, imagePathList[-2])
    if not os.path.exists(outputImagePath):
        os.mkdir(outputImagePath)

        
def decideOutputFolderName():
    outputFolderName = ""
    for process in processList:
        if process == 0:
            outputFolderName += "彩度" + str(saturationRatio)
        elif process == 1:
            outputFolderName += "コントラスト" + str(contrastRatio)
        elif process == 2:
            outputFolderName += "明度" + str(brightnessRatio)
        elif process == 3:
            outputFolderName += "シャープネス" + str(sharpnessRatio)
        elif process == 4:
            outputFolderName += "大きさ" + str(expansionRatio)
        elif process == 5:
            outputFolderName += "上下反転"
        elif process == 6:
            outputFolderName += "左右反転"
        elif process == 7:
            outputFolderName += "90度回転"
        elif process == 8:
            outputFolderName += "270度回転"
    return outputFolderName

def copy(inputObject, outputObject):
    if os.path.exists(inputObject):
        shutil.copy(inputObject, outputObject)

def processing(image, coorList):
    for process in processList:
        if process == 0:
            image = changeSaturation(image, saturationRatio)
        elif process == 1:
            image= changeContrast(image, contrastRatio)
        elif process == 2:
            image = changeBrightness(image, brightnessRatio)
        elif process == 3:
            image = changeSharpness(image, sharpnessRatio)
        elif process == 4:
            image, coorList = changeSize(image, coorList, expansionRatio)
        elif process == 5:
            image, coorList = flipVertical(image, coorList)
        elif process == 6:
            image, coorList = flipHorizontal(image, coorList)
        elif process == 7:
            image, coorList = rotation90(image, coorList)
        elif process == 8:
            image, coorList = rotation270(image, coorList)
    return image, coorList

def changeSaturation(image, ratio):
    saturation = ImageEnhance.Color(image)
    return saturation.enhance(ratio)

def changeContrast(image, ratio):
    contrast = ImageEnhance.Contrast(image)
    return contrast.enhance(ratio)

def changeBrightness(image, ratio):
    brightness = ImageEnhance.Brightness(image)
    return brightness.enhance(ratio)

def changeSharpness(image, ratio):
    sharpness = ImageEnhance.Sharpness(image)
    return sharpness.enhance(ratio)

def changeSize(image, coorList, ratio):
    w, h = image.size
    resizedImage = image.resize((int(w * ratio), int(h * ratio)))
    
    resizedCoorList = []
    for oneCoorList in coorList:
        xmin = int(oneCoorList[0])
        ymin = int(oneCoorList[1])
        xmax = int(oneCoorList[2])
        ymax = int(oneCoorList[3])
        resizedCoorList.append([str(int(xmin * ratio)), str(int(ymin * ratio)), str(int(xmax * ratio)), str(int(ymax * ratio)), oneCoorList[4]])
    return resizedImage, resizedCoorList

def flipVertical(image, coorList):
    flippedImage = ImageOps.flip(image)
    
    flippedCoorList = []
    w, h = image.size
    for oneCoorList in coorList:
        xmin = int(oneCoorList[0])
        ymin = int(oneCoorList[1])
        xmax = int(oneCoorList[2])
        ymax = int(oneCoorList[3])
        flippedCoorList.append([str(xmin), str(h - ymax), str(xmax), str(h - ymin), oneCoorList[4]])
    return flippedImage, flippedCoorList

def flipHorizontal(image, coorList):
    mirrorImage = ImageOps.mirror(image)
    
    mirrorCoorList = []
    w, h = image.size
    for oneCoorList in coorList:
        xmin = int(oneCoorList[0])
        ymin = int(oneCoorList[1])
        xmax = int(oneCoorList[2])
        ymax = int(oneCoorList[3])
        mirrorCoorList.append([str(w - xmax), str(ymin), str(w - xmin), str(ymax), oneCoorList[4]])
    return mirrorImage, mirrorCoorList

def rotation90(image, coorList):
    rotatedImage = image.rotate(90, expand = True)
    
    rotatedCoorList = []
    w, h = image.size
    for oneCoorList in coorList:
        xmin = int(oneCoorList[0])
        ymin = int(oneCoorList[1])
        xmax = int(oneCoorList[2])
        ymax = int(oneCoorList[3])
        rotatedCoorList.append([str(ymin), str(w - xmax), str(ymax), str(w - xmin), oneCoorList[4]])
    return rotatedImage, rotatedCoorList

def rotation270(image, coorList):
    rotatedImage = image.rotate(270, expand = True)
    
    rotatedCoorList = []
    w, h = image.size
    for oneCoorList in coorList:
        xmin = int(oneCoorList[0])
        ymin = int(oneCoorList[1])
        xmax = int(oneCoorList[2])
        ymax = int(oneCoorList[3])
        rotatedCoorList.append([str(h - ymax), str(xmin), str(h - ymin), str(xmax), oneCoorList[4]])
    return rotatedImage, rotatedCoorList
    
    


In [69]:
# 実行するところ

In [71]:
outputFolderName = decideOutputFolderName()

imageName = r'\*\*.jpg'
if allFruits:
    imageName = r'\*\*\*.jpg'

### folder作成
for inputImageFile in glob.glob(inputImageFolderPath + imageName):
    splitInputImageFile = inputImageFile.split('\\')
    if 'Thumbs.jpg' in splitInputImageFile[-1]:
        continue
    makeFolder(outputRoot, outputFolderName, inputImageFile)

### 画像ファイル取得
error_list = []

splitinputImageFolderPath = inputImageFolderPath.split('\\')
outputImageFolderPath = os.path.join(outputRoot, outputFolderName, splitinputImageFolderPath[-3], splitinputImageFolderPath[-2], splitinputImageFolderPath[-1])
for image in glob.glob(inputImageFolderPath + imageName):
    splitImage = image.split('\\')
    if 'Thumbs.jpg' in splitImage[-1]:
        continue
    print(image)
    try:
        originalImage = Image.open(image).convert("RGB")
        originalText = image.replace("image", "annotation").replace(".jpg", ".txt")
        coorList = []
        if os.path.exists(originalText):
            with open(originalText) as f:
                for s_line in f:
                    if len(s_line.split(',')) == 1:
                        pass
                    else:
                        coorList.append(s_line.split(','))


        ### 画像水増し処理
        processedImage, processedCoorList = processing(originalImage, coorList)

        ### 出力
        outputImageFile = os.path.join(outputRoot, outputFolderName, splitImage[-5], splitImage[-4], splitImage[-3], splitImage[-2], splitImage[-1])
        processedImage.save(outputImageFile)
        outputTextFile = outputImageFile.replace("image", "annotation").replace(".jpg", ".txt")
        with open(outputTextFile, mode = 'w') as f:
            for oneCoorList in processedCoorList:
                f.write(oneCoorList[0] + "," + oneCoorList[1] + "," + oneCoorList[2] + "," + oneCoorList[3] + "," + oneCoorList[4])
    except Exception as e:
        error_list.append(e)

if error_list:
    print(error_list)
    print(len(error_list))

#TODO:trycatchは動くものの、処理が続かない。

"""
2020-09-18　追記

trycatchはfor文内に入れないとfor文を外れてcatchされます
例外処理が起こるであろう箇所をある程度決めてtrycatchで囲みましょう
"""

\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\1.sato1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\10.img10051927485.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\10_092F-005s1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\113.010705sat.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\116.20120307roji-01.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\119.satonishiki.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\11_002-00210sq0.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\125.00120012-thumb-360x360-75.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\126.maxresdefault.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\12_seikaokoku_s-fs3055.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\131.maxresdef

\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\60.20160701-3.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\62.satos1--770x513.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\64.sakuranbo2019.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\67.20120307roji-26.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\68.2017roji05.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\6_ecg2006011200001-p1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\7_001.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\8.13701407791.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\81.imgrc0064342009.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\84.saku_01_sato_1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\1_佐藤錦\85.saku_0520_tilyo2.jpg
\\17

\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\25.imgrc0070357607.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\253.gasan-top500ga.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\255.Da38uzkUwAEcXtT.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\26.20170822230821474.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\265.a4ba9c717a386b33826d5cd257f04a82.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\269.fodv0000002001_1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\27.0000073288.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\282.gasan-1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\29.gr441-i1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\292.jag14-05.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\2_月山錦\29

\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\140.taisyounisiki500g1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\143.20537.1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\144.pc-yj-och-k1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\145.slider-taisho.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\147.summer_2018_4_02.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\14_20537.2.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\15.hqdefault.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\15_main-image1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\16_070711.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\171.20110518taisilyou-11.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\3_大将錦\17_pd_20388656066956838bf

\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\120.maxresdefault.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\128.benitoltupu.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\129.004594.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\14.gr89_type_01.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\141.imgrc0061953278.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\144.sakuranbo.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\145.benishuho1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\146.001benishuho01.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\147.benis2-.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\150.saku_05_beni_1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\4_紅秀峰\151.benishuho.jpg
\\172.18.21.

\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\1.maxresdefault.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\10_3df2ef3904e08ad7686d9d6615335b0a-1-768x434.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\118.k111_18sk-nanyo500b-m.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\119.nanyo.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\11_4ac5cab087b781af436b7701fb4788e3-320x315.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\120.shouhin3.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\123.nanyou.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\125.20200704233722206.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\126.nannyou6.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\12_IMG_2944_F.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image

\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\56.tn01-296t001-01.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\5_12588_1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\6_02937_2010_0703_IMG_0078.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\7.img56151949.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\71.01050088_free_7_1.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\74.sakuranbo02.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\75.shouhin3.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\79.header.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\7_05985_2013_0719_IMG_9705.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\8.imgrc0069876454.jpg
\\172.18.21.187\aiコンテスト2020\100_開発\学習データ\混合データ\品種\image\7_さくらんぼ\5_南陽\83.fa970dbdbd.jpg
\\172.18.2